<a href="https://colab.research.google.com/github/halfbug/colab/blob/main/agent4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain and OpenAI installation**


In [1]:
try:
  import openai
  from langchain_openai import ChatOpenAI
except:
    !pip install openai
    !pip install langchain
    !pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.3 MB/s eta 0:00:00


In [2]:
import openai
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
from langchain.chat_models import ChatOpenAI
import datetime
current_date = datetime.datetime.now().date()
llm_name = "gpt-4"
# if current_date < datetime.date(2023, 9, 2):
#     llm_name = "gpt-3.5-turbo-0301"
# else:
#     llm_name = "gpt-3.5-turbo"
print(llm_name)
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.invoke("Hello world!")

gpt-4


AIMessage(content='Hello! How can I assist you today?')

# Scraped Webpage

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

def extract_availability(url):
    # Function to scrape the webpage for availability information
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
         # Search for specific keywords like "stock availability" on the webpage

        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text()
            # Search for either "stock" or "stock availability" in the text content
            # print(text_content)
            #  availability_pattern = re.compile(r"(?:stock|stock|availability|Number\sof\sItems\s*(\d+)|in\s*stock|(\w+)\s*stock\s*(\w+))", re.IGNORECASE)

            availability_pattern = re.compile(r"(?:stock|stock|availability|in\s*stock)", re.IGNORECASE)
            match = re.search(availability_pattern, text_content)
            # print(match.groups())
            if match:
                # print(match.group(0))
                return f"Availability: {match.group(0).strip()}"
        # If specific keywords not found, return "Scraped Availability: Unknown"
        return "Availability: Unknown"
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return "Availability: Unknown"

In [ ]:
import re

def extract_product_info(meta_tags):
    product_info = ""
    for meta_tag in meta_tags:
        if 'og:title' in meta_tag:
            product_info += f"\n ***Title: {meta_tag['og:title']}\n"
        if 'og:description' in meta_tag:
            product_info += f"Description: {meta_tag['og:description']}\n"
        if 'og:price:amount' in meta_tag and 'og:price:currency' in meta_tag:
            product_info += f"Price: {meta_tag['og:price:amount']} {meta_tag['og:price:currency']}\n"
        if 'og:image' in meta_tag:
                if len(meta_tag['og:image']) <= 200:  # Check if URL length is less than or equal to 200
                    product_info += f"Image URL: {meta_tag['og:image']} \n"
        if 'og:url' in meta_tag:
            product_info += f"URL: {meta_tag['og:url']} \n"
            # Call function to extract availability information
            availability_info = extract_availability(meta_tag['og:url'])
            product_info += f"{availability_info}\n"
            # Extract ASIN from URL if it's an Amazon link
            asin_match = re.search(r'/dp/(\w+)', meta_tag['og:url'])
            if asin_match:
                product_info += f"ASIN: {asin_match.group(1)} \n"
        if 'og:site_name' in meta_tag:
            product_info += f"Site Name: {meta_tag['og:site_name']} \n"
        if 'og:availability' in meta_tag:
            product_info += f"Availability: {meta_tag['og:availability']} \n"
        if 'og:type' in meta_tag:
                product_info += f"type: {meta_tag['og:type']} \n"
        else:
          product_info += f"type: text \n"
    return product_info

# Example usage:
# meta_tags = [{'apple-itunes-app': 'app-id=529356019, app-argument=https://www.crateandbarrel.com/apotheke-canvas-scented-3-wick-candle/s569504',
#  'og:image': 'https://cb.scene7.com/is/image/Crate/ApothekeCnvs3WickCandleSSF23', 'og:type': 'product', 'og:image:alt': 'Apotheke Canvas-Scented 3-Wick Candle',
#   'twitter:card': 'summary_large_image', 'twitter:title': 'Apotheke Canvas-Scented 3-Wick Candle + Reviews | Crate & Barrel', 'og:site_name': 'Crate & Barrel', 'og:availability':
#    'InStock', 'og:price:amount': '$78.00', 'author': 'Crate & Barrel', 'og:title': 'Apotheke Canvas-Scented 3-Wick Candle + Reviews | Crate & Barrel', 'og:price:currency': 'USD', 'language':
#     'English', 'og:description': 'Shop Apotheke Canvas-Scented 3-Wick Candle.  This hand-poured candle fills the room with clean, refreshing scent.  Infused with essences of crisp linen, white musk,
#      lily of the valley and dew drop, the soy-blend wax candle burns for many hours releasing a sweet, fresh fragrance.', 'og:price:standard_amount': '$78.00', 'twitter:site': '@CrateandBarrel',
#      'viewport': 'width=device-width, initial-scale=1.0', 'fb:admins': '1769841725', 'og:url': 'https://www.amazon.com/dp/B09RTPD6ZN/ref=tsm_1_fb_lk'}]
# product_info = extract_product_info(meta_tags)
# print(product_info)

# Google Search API

In [5]:
from googleapiclient.discovery import build

API_KEY = userdata.get('GOOGLE_SEARCH_API')
# os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')

service = build("customsearch", "v1", developerKey=API_KEY)
# clientProd = build("shopping", "v1", developerKey=API_KEY)

In [ ]:
def search_google(query):
    """
    Performs a custom search  using the Google Custom Search API.

    Args:
        query (str): The search query to use.
        num_results (int, optional): The number of image search results to return (default: 3).

    Returns:
        str: A string containing information about the image search results. Each result includes the image URL and its corresponding link.
    """
    # user_query= extract_text_between_hashes(query)
    results = service.cse().list(
        q=query, cx=userdata.get('GOOGLE_CSE_ID'),  # Replace with your custom search engine ID
        # searchType="image",  # Specify image search
        num=3
    ).execute()

    image_results = ""
    for item in results.get("items", []):
        print(item)
        image_url = item["pagemap"]["cse_image"][0]["src"]
        # image_results.append({"link":item["link"] ,
        #                       "title": item["title"] ,
        #                       "snippets":item["snippet"],
        #                       "image": image_url,
        #                       "meta": extract_product_info(item["pagemap"]["metatags"])})
        image_results += extract_product_info(item["pagemap"]["metatags"])
        image_results += f"image2: {image_url} \n"
        image_results += f"link: {item['link']}"

    len(image_results)
    return image_results

# Example usage:
# image_results = search_google("###Apotheke 3 wick candles collection price images###")
# image_results
# len(image_results)